This chapter gives an overview of OCaml by walking through a series of small examples that cover most of the major features of the language. This should provide a sense of what OCaml can do, without getting too deep into any one topic.

Throughout the book we’re going to use **Base**, a more full-featured and capable replacement for OCaml’s standard library. We’ll also use **utop**, a shell that lets you type in expressions and evaluate them interactively. **utop** is an easier-to-use version of OCaml’s standard toplevel (which you can start by typing ocaml at the command line). These instructions will assume you’re using **utop**, but the ordinary toplevel should mostly work fine.

--------------
## Base, Core and Core_kernel

**Base** is one of a family of three standard library replacements, each with different use-cases, each building on the last. Here’s a quick summary.

- **Base** is designed to provide the fundamentals needed from a standard library. It has a variety of basic efficient data structures like hash-tables, sets and sequences. It also defines the basic idioms for error handling and serialization, and contains well organized APIs for every basic data type from integers to lazy values. This is comes along with a minimum of external dependencies, so **Base** just takes seconds to build and install. It’s also portable, running on every platform that OCaml does, including Windows and JavaScript.
- **Core_kernel** extends **Base** with many new data structures, like heaps, and other capabilities, like types to represent times and time-zones, and support for efficient binary serializers. It’s still portable, but has many more dependencies, takes longer to build, and will add more to the size of your executables.
- **Core** is the most full-featured, extending **Core_kernel** with support for a variety of UNIX APIs, but only works on UNIX-like OSs, including Linux and macOS.

We use **Base** in this section, but you should check out **Core** and **Core_kernel**, depending on your requirements. We’ll discuss some of the additional functionality provided by Core and Core_kernel later in the book.

--------------

Before getting started, make sure you have a working OCaml installation so you can try out the examples as you read through the chapter.

# OCaml as a Calculator

Our first step is to open **Base**:
```ocaml
open Base;;
```

By opening **Base**, we make the definitions it contains available without having to reference **Base** explicitly. This is required for many of the examples in the tour and in the remainder of the book.

Now let’s try a few simple numerical calculations:
```ocaml
3 + 4;;
- : int = 7
8 / 3;;
- : int = 2
3.5 +. 6.;;
- : float = 9.5
30_000_000 / 300_000;;
- : int = 100
3 * 5 > 14;;
- : bool = true
```

By and large, this is pretty similar to what you’d find in any programming language, but a few things jump right out at you:

- We needed to type **;;** in order to tell the toplevel that it should evaluate an expression. This is a peculiarity of the toplevel that is not required in standalone programs (though it is sometimes helpful to include **;;** to improve OCaml’s error reporting, by making it more explicit where a given top-level declaration was intended to end).
- After evaluating an expression, the toplevel first prints the type of the result, and then prints the result itself.
- Function arguments are separated by spaces instead of by parentheses and commas, which is more like the UNIX shell than it is like traditional programming languages such as C or Java.
- OCaml allows you to place underscores in the middle of numeric literals to improve readability. Note that underscores can be placed anywhere within a number, not just every three digits.
- OCaml carefully distinguishes between **float**, the type for floating-point numbers, and **int**, the type for integers. The types have different literals (**6.** instead of **6**) and different infix operators (**+.** instead of **+**), and OCaml doesn’t automatically cast between these types. This can be a bit of a nuisance, but it has its benefits, since it prevents some kinds of bugs that arise in other languages due to unexpected differences between the behavior of **int** and **float**. For example, in many languages, **1 / 3** is zero, but **1 / 3.0** is a third. OCaml requires you to be explicit about which operation you’re using.

We can also create a variable to name the value of a given expression, using the let keyword. This is known as a let binding:
```ocaml
let x = 3 + 4;;
val x : int = 7
let y = x + x;;
val y : int = 14
```

After a new variable is created, the toplevel tells us the name of the variable (**x** or **y**), in addition to its type (**int**) and value (**7** or **14**).

Note that there are some constraints on what identifiers can be used for variable names. Punctuation is excluded, except for _ and ', and variables must start with a lowercase letter or an underscore. Thus, these are legal:

```ocaml
let x7 = 3 + 4;;
val x7 : int = 7
let x_plus_y = x + y;;
val x_plus_y : int = 21
let x' = x + 1;;
val x' : int = 8
```

The following examples, however, are not legal:
```ocaml
let Seven = 3 + 4;;
Characters 4-9:
Error: Unbound constructor Seven
let 7x = 7;;
Characters 4-6:
Error: Unknown modifier 'x' for literal 7x
let x-plus-y = x + y;;
Characters 6-10:
Error: Syntax error
```

This highlights that variables can’t be capitalized, can’t begin with numbers, and can’t contain dashes.

# Functions and Type Inference

The **let** syntax can also be used to define a function:
```ocaml
let square x = x * x;;
val square : int -> int = 
square 2;;
- : int = 4
square (square 2);;
- : int = 16
```

Functions in OCaml are values like any other, which is why we use the **let** keyword to bind a function to a variable name, just as we use **let** to bind a simple value like an integer to a variable name. When using **let** to define a function, the first identifier after the **let** is the function name, and each subsequent identifier is a different argument to the function. Thus, **square** is a function with a single argument.

Now that we’re creating more interesting values like functions, the types have gotten more interesting too. **int -> int** is a function type, in this case indicating a function that takes an int and returns an int. We can also write functions that take multiple arguments. (Reminder: Don’t forget **open Base**, or these examples won’t work!)  

```ocaml
let ratio x y =
  Float.of_int x /. Float.of_int y
;;
val ratio : int -> int -> float = 
ratio 4 7;;
- : float = 0.571428571428571397
```

The preceding example also happens to be our first use of modules. Here, **Float.of_int** refers to the **of_int** function contained in the **Float** module. This is different from what you might expect from an object-oriented language, where dot-notation is typically used for accessing a method of an object. Note that module names always start with a capital letter.

Modules can also be opened to make their contents available without explicitly qualifying by the module name. We did that once already, when we opened **Base** earlier. We can use that to make this code a little easier to read, both avoiding the repetition of **Float** above, and avoiding use of the slightly awkward **/.** operator. In the following example, we open the **Float.O** module, which has a bunch of useful operators and functions that are designed to be used in this kind of context. Note that this causes the standard int-only arithmetic operators to be shadowed locally.

```ocaml
let ratio x y =
  let open Float.O in
  of_int x / of_int y
;;
val ratio : int -> int -> float = 
```

Note that we used a slightly different syntax for opening the module, since we were only opening it in the local scope inside the definition of **ratio**. There’s also a more concise syntax for local opens, as you can see here.

```ocaml
let ratio x y =
  Float.O.(of_int x / of_int y)
;;
val ratio : int -> int -> float = 
```
The notation for the type-signature of a multiargument function may be a little surprising at first, but we’ll explain where it comes from when we get to function currying in Chapter 2, Multi Argument Functions. For the moment, think of the arrows as separating different arguments of the function, with the type after the final arrow being the return value. Thus, **int -> int -> float** describes a function that takes two **int** arguments and returns a **float**.

We can also write functions that take other functions as arguments. Here’s an example of a function that takes three arguments: a test function and two integer arguments. The function returns the sum of the integers that pass the test:

```ocaml
let sum_if_true test first second =
  (if test first then first else 0)
  + (if test second then second else 0)
;;
val sum_if_true : (int -> bool) -> int -> int -> int = 
```

If we look at the inferred type signature in detail, we see that the first argument is a function that takes an integer and returns a boolean, and that the remaining two arguments are integers. Here’s an example of this function in action:

```ocaml
let even x =
x % 2 = 0
;;
val even : int -> bool = 
sum_if_true even 3 4;;
- : int = 4
sum_if_true even 2 4;;
- : int = 6
```

Note that in the definition of **even**, we used **=** in two different ways: once as part of the **let** binding that separates the thing being defined from its definition; and once as an equality test, when comparing **x % 2** to **0**. These are very different operations despite the fact that they share some syntax.

## Type Inference

As the types we encounter get more complicated, you might ask yourself how OCaml is able to figure them out, given that we didn’t write down any explicit type information. 

OCaml determines the type of an expression using a technique called type inference, by which the type of an expression is inferred from the available type information about the components of that expression.

As an example, let’s walk through the process of inferring the type of **sum_if_true**:

1. OCaml requires that both branches of an **if** statement have the same type, so the expression

    **if test first then first else 0**

    requires that **first** must be the same type as **0**, and so **first** must be of type **int**. Similarly, from

    **if test second then second else 0**

    we can infer that **second** has type **int**.

2. **test** is passed **first** as an argument. Since **first** has type **int**, the input type of **test** must be **int**.

3. **test first** is used as the condition in an **if** statement, so the return type of **test** must be **bool**.

4. The fact that **+** returns **int** implies that the return value of **sum_if_true** must be **int**.

Together, that nails down the types of all the variables, which determines the overall type of **sum_if_true**.

Over time, you’ll build a rough intuition for how the OCaml inference engine works, which makes it easier to reason through your programs. You can also make it easier to understand the types of a given expression by adding explicit type annotations. These annotations don’t change the behavior of an OCaml program, but they can serve as useful documentation, as well as catch unintended type changes. They can also be helpful in figuring out why a given piece of code fails to compile.

Here’s an annotated version of **sum_if_true**:

```ocaml
let sum_if_true (test : int -> bool) (x : int) (y : int) : int =
  (if test x then x else 0)
  + (if test y then y else 0)
;;
val sum_if_true : (int -> bool) -> int -> int -> int = 
```

In the above, we’ve marked every argument to the function with its type, with the final annotation indicating the type of the return value. Such type annotations can be placed on any expression in an OCaml program:

## Inferring Generic Types



# Tuples, Lists, Options, and Pattern Matching
## Tuples
## Lists
### THE LIST MODULE
### CONSTRUCTING LISTS WITH ::
### LIST PATTERNS USING MATCH
### RECURSIVE LIST FUNCTIONS
## Options
# Records and Variants
# Imperative Programming
## Arrays
## Mutable Record Fields
## Refs
## For and While Loops
# A Complete Program
## Compiling and Running
# Where to Go from Here